### Gruppo 5.a: Castagnacci Giulia, Giordano Elisabetta

### Analisi pointInPolygonClassification

In [1]:

using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


### Dati in input

In [2]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);                                          #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);                               #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];                                                   #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

### Versione iniziale pointInPolygonClassification

Funzione utilizzata per la classificazione dei punti di un poligono. In particolare, identifica se un punto si trova all'interno, all'esterno o sul bordo di un poligono, richiama la setTile per dirlo.

In [3]:
function pointInPolygonClassification(V,EV)
    function pointInPolygonClassification0(pnt)
        x,y = pnt
        xmin,xmax,ymin,ymax = x,x,y,y
        tilecode = setTile([ymax,ymin,xmax,xmin])
        count,status = 0,0

        for (k,edge) in enumerate(EV)
            p1,p2 = V[:,edge[1]],V[:,edge[2]]
            (x1,y1),(x2,y2) = p1,p2
            c1,c2 = tilecode(p1),tilecode(p2)
            c_edge, c_un, c_int = c1⊻c2, c1|c2, c1&c2

            if (c_edge == 0) & (c_un == 0) return "p_on"
            elseif (c_edge == 12) & (c_un == c_edge) return "p_on"
            elseif c_edge == 3
                if c_int == 0 return "p_on"
                elseif c_int == 4 count += 1 end
            elseif c_edge == 15
                x_int = ((y-y2)*(x1-x2)/(y1-y2))+x2
                if x_int > x count += 1
                elseif x_int == x return "p_on" end
            elseif (c_edge == 13) & ((c1==4) | (c2==4))
                    crossingTest(1,2,status,count)
            elseif (c_edge == 14) & ((c1==4) | (c2==4))
                    crossingTest(2,1,status,count)
            elseif c_edge == 7 count += 1
            elseif c_edge == 11 count = count
            elseif c_edge == 1
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(1,2,status,count) end
            elseif c_edge == 2
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(2,1,status,count) end
            elseif (c_edge == 4) & (c_un == c_edge) return "p_on"
            elseif (c_edge == 8) & (c_un == c_edge) return "p_on"
            elseif c_edge == 5
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(1,2,status,count) end
            elseif c_edge == 6
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(2,1,status,count) end
            elseif (c_edge == 9) & ((c1==0) | (c2==0)) return "p_on"
            elseif (c_edge == 10) & ((c1==0) | (c2==0)) return "p_on"
            end
        end
        if (round(count)%2)==1
        	return "p_in"
        else
        	return "p_out"
        end
    end
    return pointInPolygonClassification0
end

pointInPolygonClassification (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione iniziale

In [4]:
@btime pointInPolygonClassification(V,EV)

  155.000 ns (1 allocation: 32 bytes)


(::var"#pointInPolygonClassification0#9"{Matrix{Float64}, Vector{Vector{Int64}}}) (generic function with 1 method)

In [5]:
@code_warntype pointInPolygonClassification(V,EV)

MethodInstance for 

pointInPolygonClassification(::

Matrix{Float64}, ::Vector{Vector{Int64}})
  from 

pointInPolygonClassification(V, EV) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi_pointInPolygonClassification.ipynb:1
Arguments
  #self#

::Core.Const(pointInPolygonClassification)
  V::Matrix{Float64}
  EV::Vector{Vector{Int64}}
Locals
  pointInPolygonClassification0::var"#pointInPolygonClassification0#9"{Matrix{Float64}, Vector{Vector{Int64}}}
Body::var"#pointInPolygonClassification0#9"{Matrix{Float64}, Vector{Vector{Int64}}}


...

 %1 

= Main.:(var"#pointInPolygonClassification0#9"

)::Core.Const(var"#pointInPolygonClassification0#9")
│   %2 = 

Core.typeof(V)::Core.Const(Matrix{Float64})
│   %3 = Core.typeof(EV)::Core.Const(Vector{Vector{Int64}})
│   %4 = Core.apply_type(

%1, %2, %3)::Core.Const(var"#pointInPolygonClassification0#9"{Matrix{Float64}, Vector{Vector{Int64}}})
│        (pointInPolygonClassification0 = %new(%4, V, EV))
└──      return pointInPolygonClassification0



In [6]:
@benchmark pointInPolygonClassification(V,EV)

BenchmarkTools.Trial: 10000 samples with 728 evaluations.
 Range (min … max):  155.082 ns …   7.598 μs  ┊ GC (min … max): 0.00% … 97.45%
 Time  (median):     172.802 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   209.198 ns ± 149.088 ns  ┊ GC (mean ± σ):  1.39% ±  2.15%

  █▄▃▄▂▄▂▃▂▁▂▁▃▁ ▁     ▅▅  ▂▃▃▁ ▂▂                           ▁▁ ▁
  ████████████████▇▇▆▅▆███████████▆▆▅▇▇▆▅▆▅▄▄▅▄▄▅▅▅▆▅▅▄▄▃▃▄▂▄██ █
  155 ns        Histogram: log(frequency) by time        425 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

### Versione parallelizzata di pointInPolygonClassification

Nella versione modificata abbiamo creato una nuova funzione per ogni edge, sostituendo i relativi if else a catena di pointInPolygonClassification con le rispettive chiamate alle nuove funzioni.

In [7]:
function edgecode1(c_int) #c_edge == 1
    if c_int == 0 return "p_on"
    elseif c_int == 4 crossingTest(1,2,status, counter) end 
end 

edgecode1 (generic function with 1 method)

In [8]:
function edgecode2(c_int) #c_edge == 2
    if c_int == 0 return "p_on"
    elseif c_int == 4 crossingTest(2,1,status, counter) end 
end 

edgecode2 (generic function with 1 method)

In [9]:
function edgecode3(c_int) #c_edge == 3
    if c_int == 0 return "p_on"
    elseif c_int == 4 counter += 1 end 
end 

edgecode3 (generic function with 1 method)

In [10]:
function edgecode4(c_un) #c_edge == 4
    if c_un == 4 return "p_on" end 
end 

edgecode4 (generic function with 1 method)

In [11]:
function edgecode5(c1,c2) #c_edge == 5
    if (c1==0) | (c2==0) return "p_on"
    else crossingTest(1,2,status, counter) end 
end 

edgecode5 (generic function with 1 method)

In [12]:
function edgecode6(c1,c2) #c_edge == 6
    if ((c1==0) | (c2==0)) return "p_on"
    else crossingTest(2,1,status, counter) end 
end 

edgecode6 (generic function with 1 method)

In [13]:
function edgecode7(counter) #c_edge == 7
    counter += 1
end 

edgecode7 (generic function with 1 method)

In [14]:
function edgecode8(c_un) #c_edge == 8
    if (c_un == 8) return "p_on" end   
end

edgecode8 (generic function with 1 method)

In [15]:
function edgecode9_10(c1,c2) #c_edge == 9/10
    if ((c1 ==0) | (c2==0)) return "p_on" end
end

edgecode9_10 (generic function with 1 method)

In [16]:
function edgecode11() #c_edge == 11
    count = count
end

edgecode11 (generic function with 1 method)

In [17]:
function edgecode12(c_un) #c_edge = 12
    if (c_un == 12 ) return "p_on" end    
end

edgecode12 (generic function with 1 method)

In [18]:
function edgecode13(c1,c2) #c_edge = 13
    if (( c1 ==4) | (c2 == 4))
        crossingTest(1,2,status, counter) end
end

edgecode13 (generic function with 1 method)

In [19]:
function edgecode14(c1,c2) #c_edge = 14
    if (( c1 ==4) | (c2 == 4))
        crossingTest(2,1,status, counter) end
end

edgecode14 (generic function with 1 method)

In [20]:
function edgecode15(x1,x2,y1,y2,x,y)
    x_int = ((y-y2)*(x1-x2)/(y1-y2))+x2
    if x_int > x counter+=1
    elseif (x_int == x) return "p_on" end
end

edgecode15 (generic function with 1 method)

In [21]:
function pointInPolygonClassification2(V,EV) 
    function pointInPolygonClassification0(pnt)
        x,y = pnt
        xmin,xmax,ymin,ymax = x,x,y,y
        tilecode = setTile([ymax,ymin,xmax,xmin])
        count,status = 0,0

        @async for (k,edge) in enumerate(EV)
            p1,p2 = V[:,edge[1]],V[:,edge[2]]
            (x1,y1),(x2,y2) = p1,p2
            c1,c2 = tilecode(p1),tilecode(p2)
            c_edge, c_un, c_int = c1⊻c2, c1|c2, c1&c2

            if (c_edge == 0) edgecode1(c_un)
            elseif (c_edge == 12) edgecode12(c_un)
            elseif (c_edge == 3) edgecode3(c_int)
            elseif (c_edge == 15) edgecode15(x1,x2,y1,y2,x,y)
            elseif (c_edge == 13) edgecode13(c1,c2)
            elseif (c_edge == 14) edgecode14(c1,c2)
            elseif (c_edge == 7) edgecode7(counter)
            elseif (c_edge == 11) edgecode11()
            elseif (c_edge == 1) edgecode1(c_int)
            elseif (c_edge == 2) edgecode2(c_int)
            elseif (c_edge == 4) edgecode4(c_un)
            elseif (c_edge == 8) edgecode8(c_un)
            elseif (c_edge == 5) edgecode5(c1,c2)
            elseif (c_edge == 6) edgecode6(c1,c2)
            elseif ((c_edge == 9) | (c_edge == 10)) edgecode9_10(c1,c2)
            end
        end
        if (round(count)%2)==1
        	return "p_in"
        else
        	return "p_out"
        end
    end
    return pointInPolygonClassification0
end

pointInPolygonClassification2 (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione parallelizzata

In [22]:
@btime pointInPolygonClassification2(V,EV)

  154.899 ns (1 allocation: 32 bytes)


(::var"#pointInPolygonClassification0#11"{Matrix{Float64}, Vector{Vector{Int64}}}) (generic function with 1 method)

In [23]:
@benchmark pointInPolygonClassification2(V,EV)

BenchmarkTools.Trial: 10000 samples with 376 evaluations.
 Range (min … max):  157.713 ns …  15.681 μs  ┊ GC (min … max): 0.00% … 98.69%
 Time  (median):     238.564 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   233.466 ns ± 215.394 ns  ┊ GC (mean ± σ):  1.24% ±  1.39%

  ▂   ▇                   ▃█               ▁                     
  █▄▄▆█▄▃▂▂▂▂▃▂▂▂▁▁▂▁▂▂▂▁▁██▃▁▁▂▂▂▂▁▁▁▁▁▁▁▃█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  158 ns           Histogram: frequency by time          398 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

## Test

In [24]:
using Test

@testset "pointInPolygonClassification Tests" begin

    (V, EV) = ([0.43145 0.596771 0.758062 1.0 0.778226 0.919353 0.879033 0.806447 0.778226 0.709677 0.596771 0.262094 0.322578 0.0 0.2379 0.161291 0.467739 0.429435 0.627999 0.627999 0.383062 0.694833 0.653221 0.544027 0.778226 0.848789 0.750707 0.627999 0.694833 0.806447; -0.0163938 0.22521 0.104412 0.325182 0.629266 0.683418 0.820882 0.725074 0.845873 0.75215 1.0 0.820882 0.629266 0.385151 0.43765 0.246033 0.466811 0.629266 0.704244 0.507207 0.275195 0.683418 0.43765 0.291323 0.199264 0.43765 0.497413 0.341841 0.259902 0.364484], Array{Int64,1}[[1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [18, 19], [19, 20], [20, 21], [21, 1], [22, 23], [24, 23], [24, 25], [25, 26], [26, 22], [27, 28], [28, 29], [29, 30], [30, 27]])
    
    classify = Lar.pointInPolygonClassification(V,EV)
    queryPoint = [0.5,0.5]
       
        @testset "pointInPolygonClassification Tests" begin
            @test Lar.pointInPolygonClassification(V,EV) isa Function
            @test pnt = [0.5,0.5] isa Array{Float64,1}
            @test classify(queryPoint)=="p_out"
            @test classify([0.5,0.75])=="p_in"
            @test classify([1.5,0.75])=="p_out"
            @test typeof(classify(queryPoint))==String
        end
    end

Test Summary:                      | Pass  Total
pointInPolygonClassification Tests |    6  

    6


Test.DefaultTestSet("pointInPolygonClassification Tests", Any[Test.DefaultTestSet("pointInPolygonClassification Tests", Any[], 6, false, false)], 0, false, false)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/screenTest/tests_pointInPolygonClassification.png?raw=true)